In [12]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg-molbace/'
dataset = GraphPropPredDataset(name = 'ogbg-molbace')


split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [13]:
print(dataset[0][0].keys())

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])


In [14]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [15]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    labels.append(label)
    num_nodes += g["num_nodes"]

100%|██████████| 1513/1513 [00:00<00:00, 32557.35it/s]


In [16]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [17]:
_node_list = sparse.csr_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

/home/jimmyzxj/miniconda3/envs/gli_env/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [18]:
from gli.utils import save_data
data = {
    "node_feats": _node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
save_data("ogbg-molbace", **data)

Save all dense arrays to ogbg-molbace.npz, including ['node_feats', 'edge', 'graph_class', 'edge_feats']
Save sparse matrix node_list to ogbg-molbace_node_list.sparse.npz


In [21]:
_node_list

<1513x51577 sparse matrix of type '<class 'numpy.float64'>'
	with 51577 stored elements in Compressed Sparse Row format>

In [19]:
for i in data.values():
    assert i is not None

In [20]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
save_data("ogbg-molbace_task",**task_data)

Save all dense arrays to ogbg-molbace_task.npz, including ['train', 'val', 'test']
